# Vox Amelior - Full Stack Colab Setup

This notebook starts ALL core services on Google Colab.

**Prerequisites:**
1. Set `NGROK_AUTHTOKEN` in Colab Secrets
2. Set `SESSION_KEY` and `USERS_DB_KEY` (random strings)
3. (Optional) Set `HF_TOKEN` for Gemma models

**Services Started:**
- API Gateway (port 8000)
- Gemma Service (port 8001)
- Transcription Service (port 8003)
- RAG Service (port 8004)

In [ ]:
# @title 1. Mount Drive & Clone Repository
import os
import sys
from google.colab import drive

drive.mount('/content/drive')

DRIVE_BASE = "/content/drive/MyDrive/Nemo_Server"
REPO_PATH = os.path.join(DRIVE_BASE, "repo")
LIB_CACHE = os.path.join(DRIVE_BASE, "lib_cache")

os.makedirs(LIB_CACHE, exist_ok=True)
os.makedirs(REPO_PATH, exist_ok=True)

if not os.path.exists(os.path.join(REPO_PATH, "services")):
    print("Cloning repository...")
    !git clone https://github.com/pruittcolon/Nemo_Server.git "$REPO_PATH"
else:
    print("Repository found on Drive.")

%cd $REPO_PATH
print(f"Working directory: {os.getcwd()}")

In [ ]:
# @title 2. Install System Dependencies
!apt-get update -qq
!apt-get install -y ffmpeg libsndfile1 -qq
print("System dependencies installed.")

In [ ]:
# @title 3. Install Python Dependencies
import os
import sys

LIB_CACHE = "/content/drive/MyDrive/Nemo_Server/lib_cache"
if LIB_CACHE not in sys.path:
    sys.path.insert(0, LIB_CACHE)

cache_files = os.listdir(LIB_CACHE) if os.path.exists(LIB_CACHE) else []

if len(cache_files) < 10:
    print("Installing dependencies to Drive cache...")
    !pip install -t "$LIB_CACHE" \
        fastapi uvicorn pydantic httpx pyngrok nest_asyncio \
        python-multipart python-jose bcrypt cryptography \
        soundfile librosa ffmpeg-python redis faiss-cpu \
        sentence-transformers llama-cpp-python \
        --quiet
    print("Dependencies cached.")
else:
    print(f"Cache found ({len(cache_files)} items).")

In [ ]:
# @title 4. Configure Environment
import os
from google.colab import userdata

# Load secrets
os.environ["NGROK_AUTHTOKEN"] = userdata.get('NGROK_AUTHTOKEN')
os.environ["SESSION_KEY"] = userdata.get('SESSION_KEY')
os.environ["USERS_DB_KEY"] = userdata.get('USERS_DB_KEY')

# Optional HF token
try:
    os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
    os.environ["HUGGINGFACE_TOKEN"] = os.environ["HF_TOKEN"]
except:
    pass

# Service URLs (localhost mode)
os.environ["GEMMA_URL"] = "http://localhost:8001"
os.environ["TRANSCRIPTION_URL"] = "http://localhost:8003"
os.environ["RAG_URL"] = "http://localhost:8004"
os.environ["EMOTION_URL"] = "http://localhost:8005"
os.environ["INSIGHTS_URL"] = "http://localhost:8010"

# Paths for services
os.environ["DB_PATH"] = "/content/drive/MyDrive/Nemo_Server/data/rag.db"
os.environ["FAISS_INDEX_PATH"] = "/content/drive/MyDrive/Nemo_Server/data/faiss_index"
os.environ["APP_INSTANCE_DIR"] = "/content/drive/MyDrive/Nemo_Server/data/instance"

# Create data directories
os.makedirs("/content/drive/MyDrive/Nemo_Server/data/faiss_index", exist_ok=True)
os.makedirs("/content/drive/MyDrive/Nemo_Server/data/instance", exist_ok=True)

# Disable strict security for demo
os.environ["TEST_MODE"] = "true"
os.environ["JWT_ONLY"] = "false"

print("Environment configured.")

In [ ]:
# @title 5. Start All Services
import threading
import time
import sys
import os

# Setup paths
REPO_PATH = "/content/drive/MyDrive/Nemo_Server/repo"
sys.path.insert(0, REPO_PATH)
sys.path.insert(0, os.path.join(REPO_PATH, "services/api-gateway"))
sys.path.insert(0, os.path.join(REPO_PATH, "services/gemma-service"))
sys.path.insert(0, os.path.join(REPO_PATH, "services/transcription-service"))
sys.path.insert(0, os.path.join(REPO_PATH, "services/rag-service"))

# Patch asyncio
import nest_asyncio
nest_asyncio.apply()

import uvicorn

# Service startup functions
def start_api_gateway():
    print("Starting API Gateway on port 8000...")
    try:
        uvicorn.run(
            "services.api-gateway.src.main:app",
            host="0.0.0.0", port=8000, log_level="warning"
        )
    except Exception as e:
        print(f"API Gateway error: {e}")

def start_gemma_service():
    print("Starting Gemma Service on port 8001...")
    try:
        uvicorn.run(
            "services.gemma-service.src.main:app",
            host="0.0.0.0", port=8001, log_level="warning"
        )
    except Exception as e:
        print(f"Gemma Service error: {e}")

def start_transcription_service():
    print("Starting Transcription Service on port 8003...")
    try:
        uvicorn.run(
            "services.transcription-service.src.main:app",
            host="0.0.0.0", port=8003, log_level="warning"
        )
    except Exception as e:
        print(f"Transcription Service error: {e}")

def start_rag_service():
    print("Starting RAG Service on port 8004...")
    try:
        uvicorn.run(
            "services.rag-service.src.main:app",
            host="0.0.0.0", port=8004, log_level="warning"
        )
    except Exception as e:
        print(f"RAG Service error: {e}")

# Start services in threads (staggered to reduce memory spike)
services = [
    ("RAG", start_rag_service),
    ("Gemma", start_gemma_service),
    ("Transcription", start_transcription_service),
    ("API Gateway", start_api_gateway),
]

threads = []
for name, func in services:
    t = threading.Thread(target=func, daemon=True, name=name)
    t.start()
    threads.append(t)
    print(f"  [{name}] thread started")
    time.sleep(3)  # Stagger startup

print("\nAll services starting. Waiting 10s for initialization...")
time.sleep(10)
print("Services ready.")

In [ ]:
# @title 6. Create Public URL
import os
from pyngrok import ngrok

ngrok.set_auth_token(os.environ["NGROK_AUTHTOKEN"])

public_url = ngrok.connect(8000).public_url
print("")
print("=" * 50)
print("FULL STACK DEPLOYMENT SUCCESSFUL")
print("=" * 50)
print("")
print(f"Access your app at: {public_url}/ui/login.html")
print("")
print("Services running:")
print("  - API Gateway: port 8000")
print("  - Gemma AI: port 8001")
print("  - Transcription: port 8003")
print("  - RAG Search: port 8004")
print("")

In [ ]:
# @title 7. Health Check (Optional)
import requests

services_to_check = [
    ("API Gateway", "http://localhost:8000/health"),
    ("Gemma", "http://localhost:8001/health"),
    ("Transcription", "http://localhost:8003/health"),
    ("RAG", "http://localhost:8004/health"),
]

print("Service Health Check:")
for name, url in services_to_check:
    try:
        r = requests.get(url, timeout=5)
        status = "OK" if r.status_code == 200 else f"Error ({r.status_code})"
    except Exception as e:
        status = f"Failed ({type(e).__name__})"
    print(f"  {name}: {status}")